### Read the dataset in from Kaggle

In [13]:
import json

! pip install kaggle
! mkdir /root/.kaggle/

# token = {"username":"tarvitepandi","key":"174a0fa3ca8c29376c1ba61ed9087117"}
token = {"username": "stensalumets", "key": "87f48d7a114c6e2d96984b04f6eaa078"}

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

! chmod 600 /root/.kaggle/kaggle.json

! kaggle competitions download -c nlp-getting-started


mkdir: cannot create directory ‘/root/.kaggle/’: File exists
train.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


### Vectorize text

Create unique vectors for each word, add them as unique features.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer


countvectorizer=CountVectorizer()
train_vectors = countvectorizer.fit_transform(train['text'])
train_vectors_df = pd.DataFrame(data=train_vectors.toarray(), columns=countvectorizer.get_feature_names())
train_target=train['target']

test_vectors = countvectorizer.transform(test['text'])
test_vectors_df = pd.DataFrame(data = test_vectors.toarray(), columns = countvectorizer.get_feature_names())

# train_vectors #columns = different words vectorized, row=each text field
# train_target #target only
# countvectorizer.vocabulary_ #word counts, len(wordcount)==train_vectors.shape[1]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Split into train,val,test

In [ ]:
from sklearn.model_selection import train_test_split

print(len(train), "\n")
x_train, x_val, y_train, y_val = train_test_split(train_vectors_df, train_target, test_size=round(len(train)*0.3), random_state=11)

print("Shape of train set:")
print(x_train.shape)
print(y_train.shape)


x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=round(len(x_val)/3), random_state=11)

print("Shape of validation set:")
print(x_val.shape)
print(y_val.shape)
print("Shape of test set:")
print(x_test.shape)
print(y_test.shape)

7613 

Shape of train set:
(5329, 21637)
(5329,)
Shape of validation set:
(1523, 21637)
(1523,)
Shape of test set:
(761, 21637)
(761,)


###KNN

In [4]:
from sklearn.model_selection import cross_val_score

In [18]:
from sklearn.neighbors import KNeighborsClassifier

model=KNeighborsClassifier()
print(f"Up to {max(cross_val_score(model, train_vectors, train_target, cv=10, scoring='f1'))*100}% correct.")

Up to 40.625% correct.


In [19]:
model.fit(train_vectors,train_target)
proba_predict = pd.DataFrame({'id':pd.read_csv("test.csv")['id']})
proba_predict['target'] = [0 if x < 0.5 else 1 for x in model.predict(test_vectors)]

proba_predict
proba_predict.to_csv('KNN5.csv', index=False)

! kaggle competitions submit -c nlp-getting-started -f KNN5.csv -m "KNN5 attempt"

100% 22.2k/22.2k [00:01<00:00, 13.6kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets

In [20]:
model=KNeighborsClassifier(n_neighbors=10)
print(f"Up to {max(cross_val_score(model, train_vectors, train_target, cv=10, scoring='f1'))*100}% correct.")

Up to 23.83419689119171% correct.


In [21]:
model.fit(train_vectors,train_target)
proba_predict = pd.DataFrame({'id':pd.read_csv("test.csv")['id']})
proba_predict['target'] = [0 if x < 0.5 else 1 for x in model.predict(test_vectors)]

proba_predict
proba_predict.to_csv('KNN10.csv', index=False)

! kaggle competitions submit -c nlp-getting-started -f KNN10.csv -m "KNN10 attempt"

100% 22.2k/22.2k [00:01<00:00, 13.6kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets

### Linear Regressor

In [9]:
from sklearn.linear_model import RidgeClassifier

model=RidgeClassifier()
print(f"Up to {max(cross_val_score(model, train_vectors, train_target, cv=10, scoring='f1'))*100}% correct.")

Up to 69.10112359550563% correct.


In [10]:
model.fit(train_vectors,train_target)
proba_predict = pd.DataFrame({'id':pd.read_csv("test.csv")['id']})
proba_predict['target'] = [0 if x < 0.5 else 1 for x in model.predict(test_vectors)]

proba_predict
proba_predict.to_csv('Linear_Regressor.csv', index=False)

! kaggle competitions submit -c nlp-getting-started -f Linear_Regressor.csv -m "Linear Regressor attempt"

100% 22.2k/22.2k [00:01<00:00, 13.1kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets

###Logistic Regressor

In [ ]:
from sklearn.linear_model import LogisticRegression

model=LogisticRegression(max_iter=1000)
print(f"Up to {max(cross_val_score(model, train_vectors, train_target, cv=10, scoring='f1'))*100}% correct.")

Up to 72.96898079763663% correct.


In [ ]:
model.fit(train_vectors,train_target)
proba_predict = pd.DataFrame({'id':pd.read_csv("test.csv")['id']})
proba_predict['target'] = [0 if x < 0.5 else 1 for x in model.predict(test_vectors)]

proba_predict
proba_predict.to_csv('Logistic_Regressor.csv', index=False)

! kaggle competitions submit -c nlp-getting-started -f Logistic_Regressor.csv -m "Logistic Regressor attempt"

100% 22.2k/22.2k [00:00<00:00, 29.7kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets

###Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier()
print(f"Up to {max(cross_val_score(model, train_vectors, train_target, cv=10, scoring='f1'))*100}% correct.")

Up to 70.97902097902097% correct.


In [ ]:
model.fit(train_vectors,train_target)
proba_predict = pd.DataFrame({'id':pd.read_csv("test.csv")['id']})
proba_predict['target'] = [0 if x < 0.5 else 1 for x in model.predict(test_vectors)]

proba_predict
proba_predict.to_csv('Random_Forest.csv', index=False)

! kaggle competitions submit -c nlp-getting-started -f Random_Forest.csv -m "Random Forest attempt"

100% 22.2k/22.2k [00:01<00:00, 20.3kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets

###Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier

model=DecisionTreeClassifier()
# print(f"Up to {max(cross_val_score(model, train_vectors, train_target, cv=10, scoring='f1'))*100}% correct.")

In [17]:
model.fit(train_vectors,train_target)
proba_predict = pd.DataFrame({'id':pd.read_csv("test.csv")['id']})
proba_predict['target'] = [0 if x < 0.5 else 1 for x in model.predict(test_vectors)]

proba_predict
proba_predict.to_csv('Decision_Tree.csv', index=False)

! kaggle competitions submit -c nlp-getting-started -f Decision_Tree.csv -m "Decision Tree attempt"

100% 22.2k/22.2k [00:01<00:00, 13.2kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets

### Gradient Boosting

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

model=GradientBoostingClassifier()
print(f"Up to {max(cross_val_score(model, train_vectors, train_target, cv=10, scoring='f1'))*100}% correct.")

Up to 64.81802426343154% correct.


In [14]:
model.fit(train_vectors,train_target)
proba_predict = pd.DataFrame({'id':pd.read_csv("test.csv")['id']})
proba_predict['target'] = [0 if x < 0.5 else 1 for x in model.predict(test_vectors)]

proba_predict
proba_predict.to_csv('Gradient_Boosting.csv', index=False)

! kaggle competitions submit -c nlp-getting-started -f Gradient_Boosting.csv -m "Gradient Boosting attempt"

100% 22.2k/22.2k [00:01<00:00, 13.2kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets

### MLP Classifier

In [6]:
from sklearn.neural_network import MLPClassifier

model=MLPClassifier()
# print(f"Up to {max(cross_val_score(model, train_vectors, train_target, cv=10, scoring='f1'))*100}% correct.")

In [8]:
model.fit(train_vectors,train_target)
proba_predict = pd.DataFrame({'id':pd.read_csv("test.csv")['id']})
proba_predict['target'] = [0 if x < 0.5 else 1 for x in model.predict(test_vectors)]

proba_predict
proba_predict.to_csv('MLP_Classifier.csv', index=False)

! kaggle competitions submit -c nlp-getting-started -f MLP_Classifier.csv -m "MLP_Classifier attempt"

100% 22.2k/22.2k [00:01<00:00, 13.2kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets